In [2]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')

In [16]:
movie_data = pd.read_csv('Data_Files//movies_metadata.csv')
movie_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [17]:
# Cleanig the genres column and making it concise for later use
movie_data['genres'] = movie_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [18]:
movie_data['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

In [22]:
# converting vote_count and vote_average to integer and finding the mean of vote averages, m is the quantile
vote_counts = movie_data[movie_data['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movie_data[movie_data['vote_average'].notnull()]['vote_average'].astype('int')
average = vote_averages.mean()
print(average)
m = vote_counts.quantile(0.95)
print(m)

5.244896612406511
434.0


In [23]:
# creating a year column and extracting year from the date column 
movie_data['year'] = pd.to_datetime(movie_data['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [24]:
movie_data['year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45461     NaT
45462    2011
45463    2003
45464    1917
45465    2017
Name: year, Length: 45466, dtype: object

In [41]:
links = pd.read_csv('Data_Files//links.csv')
credits = pd.read_csv('Data_Files//credits.csv')
keywords = pd.read_csv('Data_Files//keywords.csv')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [30]:
movie_data = movie_data.drop([19730, 29503, 35587])

In [31]:
movie_data['id'] = movie_data['id'].astype('int')

In [32]:
# lmd = large-movie-data
lmd = movie_data[movie_data['id'].isin(links)]
lmd.shape

(45463, 25)

In [33]:
# lmd['tagline'] = lmd['tagline'].fillna('')
# lmd['description'] = lmd['overview'] + lmd['tagline']
# lmd['description'] = lmd['description'].fillna('')

In [34]:
# lmd['description'][1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.Roll the dice and unleash the excitement!"

In [35]:
# # here we use tfidf to analyze the word vector for description column
# tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# tfidf_matrix = tf.fit_transform(lmd['description'])

In [36]:
# tfidf_matrix.shape

(45463, 1104495)

In [37]:
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [38]:
lmd = lmd.reset_index()
titles = lmd['title']
indices = pd.Series(lmd.index, index=lmd['title'])

In [39]:
lmd

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...
3,3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom..."
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT,Rising and falling between a man and woman.Ris...
45459,45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,,Century of Birthing,False,9.0,3.0,2011,An artist struggles to finish his work while a...
45460,45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,"When one of her hits goes wrong, a professiona..."
45461,45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,...,87.0,[],Released,,Satan Triumphant,False,0.0,0.0,1917,"In a small town live two brothers, one a minis..."


In [42]:
# Cleanig the keywords and credits data and making it concise for later use
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movie_data['id'] = movie_data['id'].astype('int')

In [43]:
movie_data = movie_data.merge(credits, on='id')
movie_data = movie_data.merge(keywords, on='id')

In [45]:
lmd = movie_data[movie_data['id'].isin(links)]
lmd.shape

(46628, 28)

In [47]:
# Cleanig the cast, crew column and making it concise for later use
lmd['cast'] = lmd['cast'].apply(literal_eval)
lmd['crew'] = lmd['crew'].apply(literal_eval)
lmd['keywords'] = lmd['keywords'].apply(literal_eval)
lmd['cast_size'] = lmd['cast'].apply(lambda x: len(x))
lmd['crew_size'] = lmd['crew'].apply(lambda x: len(x))

In [48]:
lmd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,Subdue,False,4.0,1.0,NaT,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 10703, 'name': 'tragic love'}]",3,9
46624,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,Century of Birthing,False,9.0,3.0,2011,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 2679, 'name': 'artist'}, {'id': 14531,...",11,6
46625,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,Betrayal,False,3.8,6.0,2003,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[],15,5
46626,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,Satan Triumphant,False,0.0,0.0,1917,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[],5,2


In [50]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [51]:
# # Cleanig the director column and making it concise for later use
lmd['director'] = lmd['crew'].apply(get_director)
lmd['cast'] = lmd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
lmd['cast'] = lmd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
lmd['keywords'] = lmd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
lmd['cast'] = lmd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
lmd['director'] = lmd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
lmd['director'] = lmd['director'].apply(lambda x: [x,x, x])


In [54]:
# performing the keyword count
key_count = lmd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
key_count.name = 'keyword'
key_count = key_count.value_counts()
key_count[:5]


woman director      3128
independent film    1942
murder              1314
based on novel       841
musical              734
Name: keyword, dtype: int64

In [58]:
# stemmer is used to mark similar words same ex. dogs and dog are the same word 
key_count = key_count[key_count > 1]
stemmer = SnowballStemmer('english')

In [60]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

lmd['keywords'] = lmd['keywords'].apply(filter_keywords)
lmd['keywords'] = lmd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
lmd['keywords'] = lmd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
lmd['soup'] = lmd['keywords'] + lmd['cast'] + lmd['director'] + lmd['genres']
lmd['soup'] = lmd['soup'].apply(lambda x: ' '.join(x))

# count vector for the metadata
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(lmd['soup'])

In [61]:
# calculating cosine_similarity of the metadata based vector
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [63]:
lmd = lmd.reset_index()
titles = lmd['title']
indices = pd.Series(lmd.index, index=lmd['title'])

In [64]:
# cleaned movie dataset
lmd

,level_0,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
0,0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,...,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[toy, boy, friendship, friend]",13,106,"[johnlasseter, johnlasseter, johnlasseter]",toy boy friendship friend tomhanks timallen do...
1,1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,...,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",[],26,16,"[joejohnston, joejohnston, joejohnston]",robinwilliams jonathanhyde kirstendunst joejoh...
2,2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,...,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...",[fish],7,4,"[howarddeutch, howarddeutch, howarddeutch]",fish waltermatthau jacklemmon ann-margret howa...
3,3,3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,...,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...",[],10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]",whitneyhouston angelabassett lorettadevine for...
4,4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,...,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[daughter, gynecologist]",12,7,"[charlesshyer, charlesshyer, charlesshyer]",daughter gynecologist stevemartin dianekeaton ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,46623,46623,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,...,4.0,1.0,NaT,"[leilahatami, kouroshtahami, elhamkorda]","[{'credit_id': '5894a97d925141426c00818c', 'de...",[],3,9,"[hamidnematollah, hamidnematollah, hamidnemato...",leilahatami kouroshtahami elhamkorda hamidnema...
46624,46624,46624,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,...,9.0,3.0,2011,"[angelaquino, perrydizon, hazelorencio]","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[artist, play, pinoy]",11,6,"[lavdiaz, lavdiaz, lavdiaz]",artist play pinoy angelaquino perrydizon hazel...
46625,46625,46625,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,...,3.8,6.0,2003,"[erikaeleniak, adambaldwin, juliedupage]","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[],15,5,"[markl.lester, markl.lester, markl.lester]",erikaeleniak adambaldwin juliedupage markl.les...
46626,46626,46626,False,NaN,0,[],NaN,227506,tt0008536,en,...,0.0,0.0,1917,"[iwanmosschuchin, nathalielissenko, pavelpavlov]","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[],5,2,"[yakovprotazanov, yakovprotazanov, yakovprotaz...",iwanmosschuchin nathalielissenko pavelpavlov y...


In [3]:
reader = Reader()

ratings = pd.read_csv('Data_Files//ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [5]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [6]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8057  0.8053  0.8061  0.8057  0.0003  
MAE (testset)     0.6104  0.6102  0.6106  0.6104  0.0002  
Fit time          746.15  753.71  776.67  758.84  12.98   
Test time         106.20  105.93  95.89   102.68  4.80    


{'test_rmse': array([0.80571985, 0.80532833, 0.80614815]),
 'test_mae': array([0.6103733 , 0.61015395, 0.61057662]),
 'fit_time': (746.1520004272461, 753.7127070426941, 776.6682245731354),
 'test_time': (106.19903898239136, 105.93161129951477, 95.89438819885254)}

In [7]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [12]:
svd.predict(10, 3020, 4)

Prediction(uid=10, iid=3020, r_ui=4, est=4.125189161173455, details={'was_impossible': False})

In [77]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

id_map = pd.read_csv('Data_Files//links.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(lmd[['title', 'id']], on='id').set_index('title')
indices_map = id_map.set_index('id')


In [80]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = lmd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(30)

In [81]:
hybrid(3000, 'The Godfather')                # this are the best possible recommendations based on a particular user

,title,vote_count,vote_average,year,id,est
1199,The Godfather: Part II,3418.0,8.3,1974,240,4.246934
1186,Apocalypse Now,2112.0,8.0,1979,28,4.102027
3635,The Conversation,377.0,7.5,1974,592,3.847925
13474,Chinese Coffee,16.0,6.3,2000,49511,3.670107
5309,The Gambler,21.0,7.0,1974,44800,3.593011
8911,Rumble Fish,141.0,6.8,1983,232,3.347481
1934,The Godfather: Part III,1589.0,7.1,1990,242,3.323440
1614,The Rainmaker,239.0,6.7,1997,11975,3.302125
4470,Tucker: The Man and His Dream,71.0,6.5,1988,28176,3.254798
1312,Dracula,1087.0,7.1,1992,6114,3.245694
